In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

import sys
import importlib

sys.path.append(r"C:\Users\mattm\OneDrive\Desktop\Research\Projects\Triangle Lattice\Jupyter Notebooks\8Q_Triangle_Lattice_v1")

import correlation_measurements.src_correlation_measurement
importlib.reload(correlation_measurements.src_correlation_measurement)
from correlation_measurements.src_correlation_measurement import RampOscillationShotsMeasurement, generate_ramp_double_jump_correlations_filename

import src.src_current_measurement
importlib.reload(src.src_current_measurement);
from src.src_current_measurement import CurrentMeasurementCalibration, generate_current_calibration_filename, acquire_data, generate_ramp_beamsplitter_correlations_filename, generate_ramp_beamsplitter_correlations_clean_filename



In [ ]:
def post_select_counts(measurement, readout_pair_1, readout_pair_2, post_select=True):
     ### plot post selected data

    counts = measurement.get_counts()
    times = measurement.get_times()

    num_qubits = measurement.get_num_qubits()
    num_particles = 4


    bitstrings = list(product([0,1], repeat=num_qubits))
    bitstrings_n_particles = []

    counts_n_particles = []
    for i, bitstring in enumerate(bitstrings):
        if sum(bitstring) == num_particles or not post_select:
            bitstrings_n_particles.append(bitstring)
            counts_n_particles.append(counts[i, :])

    counts_n_particles = np.array(counts_n_particles, dtype=float)
    counts_n_particles_average = np.array(counts_n_particles, dtype=float)

    for j in range(counts_n_particles_average.shape[-1]):
        counts_n_particles_average[:,j] = counts_n_particles_average[:,j] / np.sum(counts_n_particles_average[:,j])

    counts_n_particles_average = np.array(counts_n_particles, dtype=float)
    counts_n_particles_average /= np.sum(counts_n_particles_average, axis=0)

    
    populations_post_selected = np.zeros((num_qubits, counts_n_particles_average.shape[-1]))
    for i in range(len(bitstrings_n_particles)):
        for j in range(num_qubits):
            if bitstrings_n_particles[i][j] == 1:
                populations_post_selected[j, :] += counts_n_particles_average[i, :]

    total_population = np.sum(populations_post_selected, axis=0)

    population_differences_post_selected = np.zeros((4, counts_n_particles.shape[-1]))
    population_differences_post_selected[0, :] = populations_post_selected[readout_pair_1[1], :] - populations_post_selected[readout_pair_1[0], :]
    population_differences_post_selected[1, :] = populations_post_selected[readout_pair_2[1], :] - populations_post_selected[readout_pair_2[0], :]

    return populations_post_selected, counts_n_particles, bitstrings_n_particles, 

def get_post_selected_covariance(measurement, populations_post_selected, counts_n_particles, bitstrings_n_particles, readout_pair_1, readout_pair_2, plot_individual_terms=False):

    times = measurement.get_times()

    covariance_post_selected = np.zeros((8, 8, counts_n_particles.shape[-1]))

    for i in range(len(bitstrings_n_particles)):
        for index_1 in range(covariance_post_selected.shape[0]):
            for index_2 in range(index_1, covariance_post_selected.shape[1]):
                if bitstrings_n_particles[i][index_1] == 1 and bitstrings_n_particles[i][index_2] == 1:
                    covariance_post_selected[index_1, index_2, :] += counts_n_particles[i, :]/np.sum(counts_n_particles, axis=0)
                

    for i in range(covariance_post_selected.shape[0]):
        for j in range(covariance_post_selected.shape[1]):
            if i == j:
                covariance_post_selected[i, j, :] = populations_post_selected[i, :] * (1 - populations_post_selected[i, :])
            elif i > j:
                covariance_post_selected[i, j, :] = covariance_post_selected[j, i, :]
            else:
                covariance_post_selected[i, j, :] -= populations_post_selected[i, :] * populations_post_selected[j, :]



    
    n3n4 = covariance_post_selected[readout_pair_2[0], readout_pair_2[1], :]
    n2n4 = covariance_post_selected[readout_pair_1[1], readout_pair_2[1], :]
    n2n3 = covariance_post_selected[readout_pair_1[1], readout_pair_2[0], :]
    n1n4 = covariance_post_selected[readout_pair_1[0], readout_pair_2[1], :]
    n1n3 = covariance_post_selected[readout_pair_1[0], readout_pair_2[0], :]
    n1n2 = covariance_post_selected[readout_pair_1[0], readout_pair_1[1], :]


    if plot_individual_terms:
        fig, axes = plt.subplots(3, 2, figsize=(10, 12), sharex=True)
        axes = axes.flatten()

        n_terms = [n1n2, n1n3, n1n4, n2n3, n2n4, n3n4]

        labels = []
        for index_1 in readout_pair_1 + readout_pair_2:
            for index_2 in readout_pair_1 + readout_pair_2:
                if index_2 > index_1:
                    labels.append(f'n{index_1+1}n{index_2+1}')



        for ax, term, label in zip(axes, n_terms, labels):
            ax.plot(times, term, 'o--')
            ax.set_title(f'Population {label}')
            ax.set_ylabel('Population')
            ax.grid(True, alpha=0.3) 

        plt.show()


    covariance_sum = n1n3 - n1n4 - n2n3 + n2n4

    return covariance_post_selected, covariance_sum


def plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=None, ylim=None):
   
    times = measurement.get_times()

    plt.plot(times, covariance_sum, 'o--')
    plt.xlabel('Time (ns)')
    plt.ylabel('Current Correlation')

    if ylim is not None:
        plt.ylim(*ylim)

    if beamsplitter_time is not None:
        plt.axvline(beamsplitter_time, color='r', alpha=0.4, linestyle='--', label='Beamsplitter Time')

    correlator_symbol = f'$\\langle j_{{{readout_pair_1[0]+1}{readout_pair_1[1]+1}}}j_{{{readout_pair_2[0]+1}{readout_pair_2[1]+1}}}\\rangle$ '
    plt.title(f'Post-Selected Current Correlations {correlator_symbol} for 8 qubits')
    plt.show()




In [ ]:
name_to_filename = {}

# 9/24/25 - 8Q
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '24', '16', '53', '27') # best
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '24', '16', '59', '42')


# 9/26/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '26', '15', '59', '52')
# name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '26', '16', '58', '08')

# 9/29/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '09', '29', '15', '40','31')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '29', '16', '06', '09')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '09', '29', '16', '45', '51')
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '09', '29', '17', '38', '37')

# 9/30/25 - 8Q
# check population oscillations from this measurement
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '09', '30', '11', '43', '58')

# 10/02/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '02', '11', '00', '43')
# name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '02', '13', '24', '29')

# 10/03/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '10', '03', '12', '51', '14')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '03', '16', '51', '48')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '03', '18', '06', '23')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '03', '20', '37', '51')


# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '10', '03', '20', '48', '53')

# 10/04/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '10', '04', '14', '37', '49')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '04', '14', '49', '41')
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '04', '17', '00', '35')

# 10/06/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '06', '12', '25', '36')
# name_to_filename['4P8Q_1234'] = generate_ramp_double_jump_correlations_filename('2025', '10', '06', '14', '24', '30')
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '06', '15', '50', '31')
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '06', '16', '13', '24')
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '06', '16', '45', '45')

# 10/07/25 - 8Q
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '07', '13', '06', '54') # resonant holes
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '07', '13', '10', '46') # off resonant holes

# name_to_filename['4P8Q_test'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '07', '16', '38', '53') # test

# 10/08/25 - 8Q
# name_to_filename['4P8Q_1278'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '08', '12', '10', '34') # ramp near sweet spot 

# 10/09/25 - 8Q
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '09', '10', '53', '04') # ramp near sweet spot 
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '11', '08', '50') # ramp near sweet spot 
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '09', '11', '24', '13') # ramp near sweet spot 
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '11', '21', '33') # ramp near sweet spot 


name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '09', '13', '08', '13') # ramp near sweet spot 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '14', '06', '28') # ramp near sweet spot 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '09', '15', '11', '43') # ramp near sweet spot 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '15', '15', '20') # ramp near sweet spot 


name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '15', '28', '54') # more shots 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '15', '49', '15') # more shots 


name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '09', '18', '28', '53') # ramp near sweet spot 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_clean_filename('2025', '10', '09', '18', '34', '07') # ramp near sweet spot 


name_to_filename['4P8Q_1254'] = generate_current_calibration_filename('2025', '10', '09', '18', '59', '20') # ramp near sweet spot 


# 10/10/25

name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '14', '27', '14') 
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '14', '32', '53') 

# testing various things
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '21', '06', '57') 
# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '21', '11', '32') 
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '21', '25', '11') 
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '10', '21', '35', '07') 


# 10/24/25
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '24', '14', '53', '35')


# 10/27/25 
# trying to optimize 12-54 correlations
# name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '27', '13', '59', '30')

name_to_filename['4P8Q_1254'] = generate_current_calibration_filename('2025', '10', '27', '17', '57', '08') 


# 10/28/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '28', '18', '42', '01')
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '28', '18', '45', '46')


name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '28', '18', '58', '49')
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '28', '19', '02', '25')


# 10/29/25

# name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '29', '14', '49', '18')
name_to_filename['4P8Q_2356'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '29', '14', '54', '20')


name_to_filename['4P8Q_2345'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '29', '17', '07', '19')


name_to_filename['4P8Q_2345'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '29', '17', '09', '39')


# 10/30/25
name_to_filename['4P8Q_2345'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '30', '12', '51', '43')

# name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '30', '15', '17', '01') # this one's bad
name_to_filename['4P8Q_1254'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '30', '15', '19', '34') # best 1245 so far


name_to_filename['4P8Q_2365'] = generate_ramp_beamsplitter_correlations_filename('2025', '10', '30', '16', '32', '27')

# 11/04/25
name_to_filename['4P8Q_1234'] = generate_ramp_beamsplitter_correlations_filename('2025', '11', '04', '11', '19', '24') # high quality 1234


name_to_measurement = {}
for name in name_to_filename:
    name_to_measurement[name] = RampOscillationShotsMeasurement(name_to_filename[name])

In [ ]:
state_even = '4P8Q_1234'
# state_even = '4P8Q_1278'
state_odd = '4P8Q_1254'

measurement_even = name_to_measurement[state_even]
measurement_odd = name_to_measurement[state_odd]

In [ ]:
### plot post selected data


post_select = True
plot_individual_terms = False

rung_to_readout_pairs = {
    2: [[0, 1], [2, 3]],
    3: [[0, 1], [4, 3]],
    4: [[0, 1], [4, 5]],
    5: [[0, 1], [6, 5]],
    6: [[0, 1], [6, 7]],
}

J_12 = 6.02*2*np.pi
rung_to_coupling = {
    2: 6.01*2*np.pi,
    3: 6.23*2*np.pi,
    4: 6.07*2*np.pi,
    5: 5.83*2*np.pi,
    6: 6.37*2*np.pi,
}

beamsplitter_offset = 5

plot_rungs = [2,4,6]
# plot_rungs = [3]
# plot_rungs = [2, 3, 4, 5, 6]
# plot_rungs = [2, 3, 4, 6]
ylim = (-0.35, 0.35)

correlations_data = []

for rung in plot_rungs:
    if not rung in rung_to_readout_pairs:
        continue
    readout_pair_1, readout_pair_2 = rung_to_readout_pairs[rung]

    print(f'Rung {rung}:')

    average_coupling = (J_12 + rung_to_coupling[rung])/2
    beamsplitter_time = abs((np.pi/4)/(average_coupling))*1e3 + beamsplitter_offset  # in ns
    print(f'beamsplitter time: {beamsplitter_time:.2f} ns')

    print(rung_to_readout_pairs[rung])
    if rung % 2 == 0:
        measurement = measurement_even
    else:
        measurement = measurement_odd
        
    populations_post_selected, counts_n_particles, bitstrings_n_particles = post_select_counts(measurement, readout_pair_1, readout_pair_2, post_select=post_select)
    covariance_post_selected, covariance_sum = get_post_selected_covariance(measurement, populations_post_selected, counts_n_particles, bitstrings_n_particles, readout_pair_1, readout_pair_2, plot_individual_terms=plot_individual_terms)
    plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim)

    # plot_select_counts(measurement, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim, post_select=post_select, plot_individual_terms=plot_individual_terms)

    current_correlation_index = np.argmin(np.abs(measurement.get_times() - beamsplitter_time))
    current_correlation_value = covariance_sum[current_correlation_index]

    print(f'Current correlation at beamsplitter time for rungs ({readout_pair_1[0]+1},{readout_pair_1[1]+1}) and ({readout_pair_2[0]+1},{readout_pair_2[1]+1}): {current_correlation_value:.4f}')
    correlations_data.append(current_correlation_value)

In [ ]:
plt.plot(populations_post_selected[0, :], 'o--', label=f'Q{1}')
plt.plot(populations_post_selected[1, :], 'o--', label=f'Q{2}')

plt.xlabel('Time (ns)')
plt.ylabel('Population')
plt.legend()
plt.show()

for rung in plot_rungs:
    plt.plot(populations_post_selected[rung, :], 'o--', label=f'Q{rung+1}')
    plt.plot(populations_post_selected[rung+1, :], 'o--', label=f'Q{rung+2}')

    plt.xlabel('Time (ns)')
    plt.ylabel('Population')
    plt.legend()
    plt.show()

In [ ]:
# correlations_data = [0.18, -0.08, 0.06, -0.03, 0.08] # 9/24/25

expected_correlations_5us = [0.185, -0.0787, 0.0787, -0.0519, 0.0779]
expected_beamsplitter_correlations_5us = [0.144, -0.0723, 0.002, -0.041, 0.036]

expected_correlations = [0.32, -0.19, 0.13, -0.10, 0.08]
expected_beamsplitter_correlations = [0.34, -0.147, 0.096, -0.08, 0.09]

distances = [1, 2, 3, 4, 5]
distances_data = np.array(plot_rungs) - 1

# plt.plot(distances, np.abs(expected_correlations), label='simulation')
# plt.plot(distances, np.abs(expected_correlations_5us), label='simulation (5 $\\mu$s)')
# plt.plot(distances, np.abs(expected_correlations), alpha=0)

# plt.plot(distances, np.abs(expected_beamsplitter_correlations), label='beamsplitter simulation', linestyle='dotted', color='green')
# plt.plot(distances, np.abs(expected_beamsplitter_correlations_5us), label='beamsplitter simulation (5 $\\mu$s)', linestyle='dotted', color='green')


plt.plot(distances_data, np.abs(correlations_data), linestyle='', marker='o', color='blue', ms=8, label='magnitude')
plt.plot(distances_data, np.abs(correlations_data), linestyle='dashed', color='blue', alpha=0.5)
plt.plot(distances_data, (correlations_data), linestyle='', marker='x', color='red', ms=8, label='data')
plt.plot(distances_data, (correlations_data), linestyle='dashed', color='red', alpha=0.5)


# plt.xlabel('rung distance')
plt.xticks(ticks=distances, labels=[f'{int(d)}' for d in distances])
plt.xlabel('distance')
plt.ylabel('Current Correlation')

plt.grid()
plt.title('Current Correlation vs Rung Distance')

plt.ylim(-0.1, 0.3)

plt.legend()
plt.show()

In [ ]:
measurement = measurement_even
measurement = measurement_odd

counts = measurement.get_counts()
times = measurement.get_times()

plt.plot(times, np.sum(counts, axis=0), label='all counts')
plt.plot(times, np.sum(counts_n_particles, axis=0), label='post-selected counts (4 particles)')
plt.xlabel('Time (ns)')
plt.ylabel('Counts')
plt.legend()
plt.title('Counts vs Time')

plt.ylim(0, np.sum(counts, axis=0).max()*1.1)
plt.show()

print(np.sum(counts, axis=0)[0])
print(np.sum(counts_n_particles, axis=0)[0])
measurement = measurement_even
measurement = measurement_odd
print(np.sum(counts_n_particles, axis=0)[0]/np.sum(counts, axis=0)[0])


In [ ]:
### get histogram of particle number
counts = measurement.get_counts()

num_qubits = measurement.get_num_qubits()
particle_numbers = np.zeros((num_qubits+1, counts.shape[-1]))

bitstrings = list(product([0,1], repeat=num_qubits))
for i in range(len(bitstrings)):
    num_particles = int(sum(bitstrings[i]))
    particle_numbers[num_particles, :] += counts[i, :]

print(sum(particle_numbers[n] for n in range(num_qubits+1)))

print(particle_numbers[4,0])

plt.figure(figsize=(10, 6))
plt.bar(range(num_qubits+1), np.mean(particle_numbers, axis=-1)/np.sum(np.mean(particle_numbers, axis=-1)))
plt.xlabel('Number of Particles')
plt.ylabel('Average Percentage')
plt.title('Histogram of Particle Numbers')
plt.xticks(range(num_qubits+1))
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
#### check other rung correlations from the 1234 measurement that we know works

measurement = name_to_measurement['4P8Q_1234']

ylim = (-0.35, 0.35)
measurement.plot_covariance_sum([0,1], [2,3])
measurement.plot_covariance_sum([2,3], [4,5])
measurement.plot_covariance_sum([4,5], [6,7])
measurement.plot_covariance_sum([2,3], [6,7])

In [ ]:
### post selection on even distances

readout_pairs_list = []
readout_pairs_list.append([[0,1], [2,3]])
readout_pairs_list.append([[2,3], [4,5]])
readout_pairs_list.append([[4,5], [6,7]])

readout_pairs_list.append([[0,1], [4,5]])
readout_pairs_list.append([[2,3], [6,7]])


measurement = name_to_measurement['4P8Q_1234']

for i in range(len(readout_pairs_list)):
    readout_pair_1, readout_pair_2 = readout_pairs_list[i]

    print(f'Rung {rung}:')

    average_coupling = (J_12 + rung_to_coupling[rung])/2
    beamsplitter_time = abs((np.pi/4)/(average_coupling))*1e3 + beamsplitter_offset  # in ns
    print(f'beamsplitter time: {beamsplitter_time:.2f} ns')

          
    populations_post_selected, counts_n_particles, bitstrings_n_particles = post_select_counts(measurement, readout_pair_1, readout_pair_2, post_select=post_select)
    covariance_post_selected, covariance_sum = get_post_selected_covariance(measurement, populations_post_selected, counts_n_particles, bitstrings_n_particles, readout_pair_1, readout_pair_2, plot_individual_terms=plot_individual_terms)
    plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=None, ylim=ylim)

    # plot_select_counts(measurement, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim, post_select=post_select, plot_individual_terms=plot_individual_terms)

    current_correlation_index = np.argmin(np.abs(measurement.get_times() - beamsplitter_time))
    current_correlation_value = covariance_sum[current_correlation_index]

    print(f'Current correlation at beamsplitter time for rungs ({readout_pair_1[0]+1},{readout_pair_1[1]+1}) and ({readout_pair_2[0]+1},{readout_pair_2[1]+1}): {current_correlation_value:.4f}')
    correlations_data.append(current_correlation_value)

In [ ]:
### post selection on odd distances

readout_pairs_list = []
# readout_pairs_list.append([[0,1], [4,3]])
# readout_pairs_list.append([[2,3], [6,5]])


### this should be equivalent to the 1234 correlations
# readout_pairs_list.append([[4,3], [6,5]])


# try 2365 now

# this should be equivalent to 1245
readout_pairs_list.append([[1,2], [5,4]])

# this should be equivalent to 1234
# readout_pairs_list.append([[4,5], [6,7]])




measurement = name_to_measurement['4P8Q_2365']
# measurement = name_to_measurement['4P8Q_2356']
# measurement = name_to_measurement['4P8Q_1254']

for i in range(len(readout_pairs_list)):
    readout_pair_1, readout_pair_2 = readout_pairs_list[i]

    print(f'Rung {rung}:')

    average_coupling = (J_12 + rung_to_coupling[rung])/2
    beamsplitter_time = abs((np.pi/4)/(average_coupling))*1e3 + beamsplitter_offset  # in ns
    print(f'beamsplitter time: {beamsplitter_time:.2f} ns')

          
    populations_post_selected, counts_n_particles, bitstrings_n_particles = post_select_counts(measurement, readout_pair_1, readout_pair_2, post_select=post_select)
    covariance_post_selected, covariance_sum = get_post_selected_covariance(measurement, populations_post_selected, counts_n_particles, bitstrings_n_particles, readout_pair_1, readout_pair_2, plot_individual_terms=plot_individual_terms)
    plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=None, ylim=ylim)

    # plot_select_counts(measurement, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim, post_select=post_select, plot_individual_terms=plot_individual_terms)

    current_correlation_index = np.argmin(np.abs(measurement.get_times() - beamsplitter_time))
    current_correlation_value = covariance_sum[current_correlation_index]

    print(f'Current correlation at beamsplitter time for rungs ({readout_pair_1[0]+1},{readout_pair_1[1]+1}) and ({readout_pair_2[0]+1},{readout_pair_2[1]+1}): {current_correlation_value:.4f}')
    correlations_data.append(current_correlation_value)

In [ ]:
### post selection on odd distances

readout_pairs_list = []

# try 2345 now

# these should be equivalent to 1234
readout_pairs_list.append([[1,2], [3,4]])
readout_pairs_list.append([[3,4], [5,6]])





measurement = name_to_measurement['4P8Q_2345']

for i in range(len(readout_pairs_list)):
    readout_pair_1, readout_pair_2 = readout_pairs_list[i]

    print(f'Rung {rung}:')

    average_coupling = (J_12 + rung_to_coupling[rung])/2
    beamsplitter_time = abs((np.pi/4)/(average_coupling))*1e3 + beamsplitter_offset  # in ns
    print(f'beamsplitter time: {beamsplitter_time:.2f} ns')

          
    populations_post_selected, counts_n_particles, bitstrings_n_particles = post_select_counts(measurement, readout_pair_1, readout_pair_2, post_select=post_select)
    covariance_post_selected, covariance_sum = get_post_selected_covariance(measurement, populations_post_selected, counts_n_particles, bitstrings_n_particles, readout_pair_1, readout_pair_2, plot_individual_terms=plot_individual_terms)
    plot_post_selected_covariance_sum(measurement, covariance_sum, readout_pair_1, readout_pair_2, beamsplitter_time=None, ylim=ylim)

    # plot_select_counts(measurement, readout_pair_1, readout_pair_2, beamsplitter_time=beamsplitter_time, ylim=ylim, post_select=post_select, plot_individual_terms=plot_individual_terms)

    current_correlation_index = np.argmin(np.abs(measurement.get_times() - beamsplitter_time))
    current_correlation_value = covariance_sum[current_correlation_index]

    print(f'Current correlation at beamsplitter time for rungs ({readout_pair_1[0]+1},{readout_pair_1[1]+1}) and ({readout_pair_2[0]+1},{readout_pair_2[1]+1}): {current_correlation_value:.4f}')
    correlations_data.append(current_correlation_value)